## csv 파일 
## 변조모델,파일이름,파일경로,라벨

In [ ]:
import os
import csv
from concurrent.futures import ThreadPoolExecutor

# 변조 모델 및 영상 ID를 추출하는 함수
def extract_model_and_video_id(filepath):
    parts = filepath.split('/')
    model = parts[-3]
    video_id = parts[-2]
    return model, video_id

# 파일 경로에서 변조 모델, 영상 ID, 파일 이름을 추출하는 함수
def extract_info_from_filepath(filepath):
    directory, filename = os.path.split(filepath)
    _, video_id = extract_model_and_video_id(directory)
    return video_id, filename

# 디렉토리에서 MP4 파일의 파일 경로를 가져오는 함수
def get_mp4_file_paths(directory):
    mp4_file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".mp4"):
                mp4_file_paths.append(os.path.join(root, file))
    return mp4_file_paths

# 파일을 실행하는 함수
def execute_file(file_path):
    # 파일 실행 명령어를 여기에 추가하세요
    pass

# 주어진 디렉토리에서 모든 MP4 파일의 파일 경로를 가져오기
subfolders = [
    "/home/alpaco/KoDF/fo",
    "/home/alpaco/KoDF/audio-driven",
    "/home/alpaco/KoDF/dffs",
    "/home/alpaco/KoDF/dfl",
    "/home/alpaco/KoDF/fsgan",
    "/home/alpaco/KoDF/origin"
]

mp4_file_paths = []
for subfolder in subfolders:
    mp4_file_paths += get_mp4_file_paths(subfolder)

# 결과를 저장할 CSV 파일 열기
with open('file_info.csv', 'w', newline='') as csv_file:
    # CSV 파일 작성기 생성
    csv_writer = csv.writer(csv_file)
    # CSV 파일의 헤더 작성
    csv_writer.writerow(['변조모델', '파일이름', '파일경로', '라벨'])

    # 멀티스레드 풀 생성
    with ThreadPoolExecutor() as executor:
        # 모든 MP4 파일에 대해 작업을 수행
        for filepath in mp4_file_paths:
            video_id, filename = extract_info_from_filepath(filepath)
            # 라벨 추가
            label = 0 if 'origin' in filepath else 1
            csv_writer.writerow([video_id, filename, filepath, label])
            # 파일 실행 작업을 멀티스레드 풀에 추가
            executor.submit(execute_file, filepath)

print("파일 경로가 file_info.csv 파일에 저장되었습니다.")

## 모든 파일 csv 리스트 만들기

In [1]:
import os
import csv
from concurrent.futures import ThreadPoolExecutor

# 디렉토리에서 MP4 파일의 파일 경로를 가져오는 함수
def get_mp4_file_paths(directory):
    mp4_file_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".mp4"):
                mp4_file_paths.append(os.path.join(root, file))
    return mp4_file_paths

subfolders = [
    "/home/alpaco/KoDF/fo",
    "/home/alpaco/KoDF/audio-driven",
    "/home/alpaco/KoDF/dffs",
    "/home/alpaco/KoDF/dfl",
    "/home/alpaco/KoDF/origin"
]

하위폴더들 = ['188276', '184963', '35706', '188264', '87f22a825d744a6a8f7f', '170669', '4ed3e16b119ac4685914']

저장할_파일 = "/home/alpaco/KoDF/하위폴더_목록.txt"

with open(저장할_파일, 'w') as 파일:
    for 폴더 in subfolders:
        for 하위폴더 in 하위폴더들:
            경로 = os.path.join(폴더, 하위폴더)
            if os.path.exists(경로) and os.path.isdir(경로):
                파일.write(경로 + '\n')

mp4_file_paths = []
for subfolder in subfolders:
    mp4_file_paths += get_mp4_file_paths(subfolder)

# 결과를 저장할 CSV 파일 열기
with open('/home/alpaco/KoDF/file_paths.csv', 'w', newline='') as csv_file:
    # CSV 파일 작성기 생성
    csv_writer = csv.writer(csv_file)
    # CSV 파일의 헤더 작성
    csv_writer.writerow(['파일이름', '파일경로'])

    # 멀티스레드 풀 생성
    with ThreadPoolExecutor() as executor:
        # 모든 MP4 파일에 대해 작업을 수행
        for filepath in mp4_file_paths:
            _, filename = os.path.split(filepath)  # 파일 이름만 추출
            # 파일 이름과 경로를 CSV 파일에 작성
            csv_writer.writerow([filename, filepath])

print("파일 경로가 file_paths.csv 파일에 저장되었습니다.")

파일 경로가 file_paths.csv 파일에 저장되었습니다.


## txt로 변환

In [2]:
import pandas as pd

# test_df.csv 파일 읽기
test_df = pd.read_csv("file_paths.csv")

# 파일경로를 저장할 텍스트 파일 경로
txt_file_path = "file_paths.txt"

# 파일경로를 텍스트 파일에 쓰기
with open(txt_file_path, "w") as txt_file:
    for file_path in test_df['파일경로']:
        txt_file.write(file_path + "\n")

print("텍스트 파일이 생성되었습니다:", txt_file_path)

텍스트 파일이 생성되었습니다: file_paths.txt


## Facecrop with MTCNN

In [ ]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor
from moviepy.editor import VideoFileClip
from mtcnn import MTCNN
from tqdm import tqdm
import logging
from pathlib import Path

# 로깅 설정
logging.basicConfig(filename="error_log.txt", level=logging.ERROR)

def process_video(video_path, input_folder, output_root_folder, detector):
    video_path = input_folder / video_path
    folder_name = video_path.stem
    parent_folder = video_path.parent.name
    superparent_folder = video_path.parent.parent.name
    output_folder = output_root_folder / superparent_folder / parent_folder / folder_name

    try:
        # 출력 폴더가 이미 존재하는지 확인
        if not output_folder.exists():
            output_folder.mkdir(parents=True)

            existing_frames = {int(p.stem) for p in output_folder.glob("*.jpg")}

            clip = VideoFileClip(str(video_path))
            fps = clip.fps
            duration = clip.duration
            frame_count = int(duration * fps)

            # 시작 프레임과 끝 프레임 설정
            start_frame = 151
            end_frame = 300

            for i in tqdm(range(start_frame, end_frame + 1), desc="프레임 처리 중"):
                if i in existing_frames:
                    continue

                frame_time = i / fps
                frame = clip.get_frame(frame_time)

                faces = detector.detect_faces(frame)
                for idx, face in enumerate(faces):
                    x, y, w, h = face['box']
                    margin_pct = 0.2
                    target_size = 224
                    cx = x + w // 2
                    cy = y + h // 2
                    new_size = int(max(w, h) * (1 + margin_pct * 2))
                    x1 = max(0, cx - new_size // 2)
                    y1 = max(0, cy - new_size // 2)
                    x2 = min(frame.shape[1], cx + new_size // 2)
                    y2 = min(frame.shape[0], cy + new_size // 2)
                    
                    face_image = frame[y1:y2, x1:x2]
                    face_image_resized = cv2.resize(face_image, (target_size, target_size))

                    filename = f'{i:05}.jpg'
                    output_file_path = output_folder / filename
                    
                    # 이미지를 RGB 순서로 저장
                    cv2.imwrite(str(output_file_path), cv2.cvtColor(face_image_resized, cv2.COLOR_RGB2BGR))

            clip.close()
            print(f"비디오 처리 완료: {video_path}")
        else:
            print(f"출력 폴더가 이미 존재합니다: {output_folder}")

    except Exception as e:
        error_msg = f"비디오 처리 중 오류 발생: {video_path}\n오류: {e}\n"
        print(error_msg)
        logging.error(error_msg)

def main():
    input_folder = Path("KODF")
    output_root_folder = Path("facecrop_kodf2")
    output_root_folder.mkdir(parents=True, exist_ok=True)

    # 파일 경로 읽어오기
    with open("file_paths.txt", "r") as file:
        video_paths = [Path(line.strip()) for line in file.readlines()]

    detector = MTCNN()

    # ThreadPoolExecutor를 사용하여 병렬로 비디오 처리 실행
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [executor.submit(process_video, video_path, input_folder, output_root_folder, detector) for video_path in video_paths]
        # 비디오 처리 완료 대기
        for future in tqdm(futures, desc="완료 대기 중"):
            future.result()

if __name__ == "__main__":
    main()


## Frame_check

In [ ]:
import os

def main():
    parent_path = "/home/alpaco/KoDF/facecrop_kodf2"
    
    subfolders = ['audio-driven', 'dffs', 'dfl', 'fo', 'origin']
    
    more_than_150_results = []  # Store results with 150 or more frames
    less_than_150_results = []  # Store results with less than 150 frames
    
    for subfolder in subfolders:
        path = os.path.join(parent_path, subfolder)
        
        if os.path.exists(path):  # Check if the path exists
            for root, dirs, files in os.walk(path):
                for file in files:
                    if file.lower().endswith('.jpg'):
                        folder_path = os.path.dirname(os.path.join(root, file))
                        jpg_count = len([f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')])
                        
                        # Append result to the appropriate list
                        if jpg_count >= 150:
                            more_than_150_results.append((subfolder, os.path.basename(folder_path), jpg_count))
                        else:
                            less_than_150_results.append((subfolder, os.path.basename(folder_path), jpg_count))
                        break  # Break after counting frames for this folder
        else:
            print(f"The path '{path}' does not exist.")
    
    return more_than_150_results, less_than_150_results

def save_to_txt(results, filename):
    with open(filename, "w") as file:
        for subfolder, folder, count in results:
            file.write(f"Subfolder: {subfolder}, Folder: {folder}, JPG Count: {count}\n")

if __name__ == "__main__":
    more_than_150_results, less_than_150_results = main()
    
    save_to_txt(more_than_150_results, "150_frames.txt")
    save_to_txt(less_than_150_results, "under_150_frames.txt")

    print("Results saved to '150_frames.txt' and 'under_150_frames.txt'.")


## under_frame_file list

In [ ]:
def generate_file_paths(txt_file, base_dir, output_file):
    with open(txt_file, 'r') as file:
        with open(output_file, 'w') as output:
            for line in file:
                # 주어진 형식에 따라서 정보 추출
                parts = line.strip().split(', ')
                subfolder = parts[0].split(': ')[1]
                folder = parts[1].split(': ')[1]

                # 파일 경로 생성
                folder_name = folder.split('_')[0]
                file_path = f"{base_dir}/{subfolder}/{folder_name}/{folder}.mp4"
                
                # 파일 경로를 새로운 텍스트 파일에 추가
                output.write(file_path + '\n')

# 기존 txt 파일 경로
txt_file_path = "/home/alpaco/KoDF/under_150_frames.txt"
# 기본 디렉토리
base_directory = "/home/alpaco/KoDF"
# 새로운 텍스트 파일 경로
output_txt_file_path = "/home/alpaco/KoDF/paths_output.txt"

# 파일 경로 생성 및 새로운 텍스트 파일에 저장
generate_file_paths(txt_file_path, base_directory, output_txt_file_path)

## 다시 생성

In [ ]:
import os
import cv2
from concurrent.futures import ThreadPoolExecutor
from moviepy.editor import VideoFileClip
from mtcnn import MTCNN
from tqdm import tqdm
import logging
from pathlib import Path

# 로깅 설정
logging.basicConfig(filename="error_log.txt", level=logging.ERROR)

def process_video(video_path, input_folder, output_root_folder, detector):
    video_path = input_folder / video_path
    folder_name = video_path.stem
    parent_folder = video_path.parent.name
    superparent_folder = video_path.parent.parent.name
    output_folder = output_root_folder / superparent_folder / parent_folder / folder_name

    try:
        # 출력 폴더가 이미 존재하는지 확인
        if not output_folder.exists():
            output_folder.mkdir(parents=True)

            existing_frames = {int(p.stem) for p in output_folder.glob("*.jpg")}

            clip = VideoFileClip(str(video_path))
            fps = clip.fps
            duration = clip.duration
            frame_count = int(duration * fps)

            # 시작 프레임과 끝 프레임 설정
            start_frame = 151
            end_frame = 300

            for i in tqdm(range(start_frame, end_frame + 1), desc="프레임 처리 중"):
                if i in existing_frames:
                    continue

                frame_time = i / fps
                frame = clip.get_frame(frame_time)

                faces = detector.detect_faces(frame)
                for idx, face in enumerate(faces):
                    x, y, w, h = face['box']
                    margin_pct = 0.2
                    target_size = 224
                    cx = x + w // 2
                    cy = y + h // 2
                    new_size = int(max(w, h) * (1 + margin_pct * 2))
                    x1 = max(0, cx - new_size // 2)
                    y1 = max(0, cy - new_size // 2)
                    x2 = min(frame.shape[1], cx + new_size // 2)
                    y2 = min(frame.shape[0], cy + new_size // 2)
                    
                    face_image = frame[y1:y2, x1:x2]
                    face_image_resized = cv2.resize(face_image, (target_size, target_size))

                    filename = f'{i:05}.jpg'
                    output_file_path = output_folder / filename
                    
                    # 이미지를 RGB 순서로 저장
                    cv2.imwrite(str(output_file_path), cv2.cvtColor(face_image_resized, cv2.COLOR_RGB2BGR))

            clip.close()
            print(f"비디오 처리 완료: {video_path}")
        else:
            print(f"출력 폴더가 이미 존재합니다: {output_folder}")

    except Exception as e:
        error_msg = f"비디오 처리 중 오류 발생: {video_path}\n오류: {e}\n"
        print(error_msg)
        logging.error(error_msg)

def main():
    input_folder = Path("KODF")
    output_root_folder = Path("facecrop_kodf2")
    output_root_folder.mkdir(parents=True, exist_ok=True)

    # 파일 경로 읽어오기
    with open("paths_output.txt", "r") as file:
        video_paths = [Path(line.strip()) for line in file.readlines()]

    detector = MTCNN()

    # ThreadPoolExecutor를 사용하여 병렬로 비디오 처리 실행
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = [executor.submit(process_video, video_path, input_folder, output_root_folder, detector) for video_path in video_paths]
        # 비디오 처리 완료 대기
        for future in tqdm(futures, desc="완료 대기 중"):
            future.result()

if __name__ == "__main__":
    main()
